In [10]:
# Importing relevant libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.ensemble import BaggingRegressor,BaggingClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, \
accuracy_score, precision_score, confusion_matrix, mean_squared_error, r2_score

#Libraries for visualizing trees
from sklearn.tree import export_graphviz 
from six import StringIO
from IPython.display import Image  
import pydotplus
import time as time
from pyearth import Earth
from sklearn.model_selection import RandomizedSearchCV, KFold


In [5]:
# Loading data
X_test = pd.read_csv('data/X_test.csv')
X_train = pd.read_csv('data/X_train.csv')
y_test = pd.read_csv('data/y_test.csv')
y_train = pd.read_csv('data/y_train.csv')

In [13]:
import warnings 
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category = FutureWarning)

# Optimizing hyperparameters
n_samples = X_train.shape[0]
n_features = X_train.shape[1]

params = {'base_estimator': [Earth(max_terms = 500, max_degree = 1),
                             Earth(max_terms = 500, max_degree = 3),
                             Earth(max_terms = 500, max_degree = 5),
                             Earth(max_terms = 500, max_degree = 8)
                            ],
          'n_estimators': [500, 1000, 1500],
          'max_samples': [0.5,1.0],
          'max_features': [0.5,1.0],
          'bootstrap': [True, False],
          'bootstrap_features': [True, False]}

cv = KFold(n_splits = 2, shuffle = True, random_state = 1)

bagging_mars_grid = RandomizedSearchCV(BaggingRegressor(random_state=1, n_jobs=-1), 
                                                         param_distributions = params,
                                                         cv = cv, 
                                                         n_iter = 50, 
                                                         n_jobs = -1, 
                                                         verbose = 1)


bagging_mars_grid.fit(X_train, y_train)

print('Best R^2 Score Through Grid Search : %.3f'%bagging_mars_grid.best_score_)
print('Best Parameters : ', bagging_mars_grid.best_params_)

Fitting 2 folds for each of 50 candidates, totalling 100 fits


/Users/lilawells/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/Users/lilawells/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/Users/lilawells/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/Users/lilawells/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_bagging.py:508: DataConversionWarning: A column-vector y was passed 